<a href="https://colab.research.google.com/github/tanyaagupta/ComplaintSystem/blob/main/textSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 42.3 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# paraphrase: This indicates that the model is trained to understand paraphrases, which are sentences or phrases that convey the same meaning but may be expressed differently. The model is trained to capture the semantic similarity between sentences and identify if they are paraphrases of each other.
# MiniLM: This refers to the architecture used for the model. In this case, it's a smaller version of the popular transformer architecture called "MiniLM." MiniLM is designed to be computationally efficient while still maintaining strong performance on various NLP tasks.
# L6: This indicates the number of transformer layers in the model. In this case, it has 6 transformer layers.

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint
import json

In [ ]:
from google.colab import files
from oauth2client.service_account import ServiceAccountCredentials


In [ ]:
uploaded = files.upload()

Saving credentials.json to credentials.json


In [ ]:
keyfile_dict = json.loads(uploaded['credentials.json'].decode('utf-8'))

# Define the scope and credentials for accessing Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_dict(keyfile_dict, scope)
client = gspread.authorize(creds)

In [ ]:
# Function to get the latest complaint and its category from Google Sheets
def get_latest_complaint_and_category():

    # Open the Google Sheet by name
    sheet = client.open("html").worksheet("Sheet2")

    # Get all complaints from the sheet
    all_complaints = sheet.get_all_records()

    # Get the latest complaint (assuming it's the last row in the sheet)
    latest_complaint = all_complaints[-1]

    return latest_complaint['Complaint'], latest_complaint['Category']

In [ ]:
# Function to get complaints with category same as input complaint
def get_complaints_by_category(category, client):
    sheet = client.open("html").worksheet("Sheet2")

    all_complaints = sheet.get_all_records()

    filtered_complaints = [{'Complaint': complaint['Complaint'], 'Solution': complaint['Solution']}
                           for complaint in all_complaints[:-1] if complaint['Category'] == category]

    return filtered_complaints

In [ ]:
def find_most_similar_complaint(input_string, complaints_with_solutions):

    # Preprocess input and complaints (lowercase, remove punctuation, etc.)
    def preprocess(text):
        return text.lower()

    input_string = preprocess(input_string)

    complaints_embeddings = []
    for item in complaints_with_solutions:
        complaint, solution = preprocess(item['Complaint']), item['Solution']
        complaint_embedding = model.encode([complaint])[0]
        complaints_embeddings.append((complaint, solution, complaint_embedding))

    # Encode input and complaints
    input_embedding = model.encode([input_string])[0]

    # Calculate similarities
    similarities = []
    for complaint, solution, complaint_embedding in complaints_embeddings:
      similarity_score = cosine_similarity(input_embedding.reshape(1, -1), complaint_embedding.reshape(1, -1))[0][0]
      similarities.append((complaint, similarity_score, solution))

    if not similarities:
        return "No similar complaint found."

    # Find the most similar complaint
    most_similar_complaint = max(similarities, key=lambda x: x[1])

    # Check if the similarity is greater than 90%
    if most_similar_complaint[1] > 0.80:
        print(most_similar_complaint[0])
        print(most_similar_complaint[1])
        return most_similar_complaint[2]
    else:
        return "No similar complaint found with a similarity score greater than 90%."

In [ ]:
# Main function
def main():

    # Get the latest complaint and its category from Google Sheets
    input_complaint, category = get_latest_complaint_and_category()

    # Fetch complaints from the same category
    complaints = get_complaints_by_category(category, client)
    print(complaints)

    if not complaints:
        print(f"No complaints found in category '{category}'")
        return

    # Find the most similar complaint
    most_similar_complaint = find_most_similar_complaint(input_complaint, complaints)

    print("\nInput Complaint:", input_complaint)
    print("Category:", category)

    if most_similar_complaint == None :
      print("No similar complaint found")
    else:
      print("Solution:", most_similar_complaint)

if __name__ == "__main__":
    main()

[{'Complaint': 'unclean washrooms', 'Solution': 'pqr'}, {'Complaint': 'unhygienic washrooms ', 'Solution': 'def2'}, {'Complaint': 'unclean washrooms', 'Solution': 'pqr2'}, {'Complaint': 'i am suffering from a chronic mental health disease and I am not being provided any sort of help from university', 'Solution': 'abc4'}]
unhygienic washrooms 
0.8554573

Input Complaint: dirty washrooms
Category: Health and Well-being Support
Solution: def2
